# CREATING AN ETL WORKFLOW FOR PREDICTING AGRICULTURAL CROP YIELDS

Import modules 

In [1]:
#from flask import Flask, redirect, url_for, render_template
import pandas as pd
import json
import requests
import re
import os
from zipfile import ZipFile
import world_bank_data as wb

# World Bank
Extract the following datasets from World Bank API: Rainfall, Agricultural Land, Population, and Ag GDP Added

Install world_bank_data if not already installed. Uncomment below. 

In [2]:
#pip install world_bank_data

Let's start with getting a list of the countries and putting the ids in a list

In [3]:
countries = wb.get_countries()
df_key = countries[['name']].rename(columns={'name': 'country'}).loc[countries.region != 'Aggregates']
df_key.reset_index(inplace=True)
key_list = df_key['id'].to_list()
df_key.head()

,id,country
0,ABW,Aruba
1,AFG,Afghanistan
2,AGO,Angola
3,ALB,Albania
4,AND,Andorra


Return a list of datasets available

In [4]:
indicators = wb.get_indicators()
indicators.head()

,name,unit,source,sourceNote,sourceOrganization,topics
id,,,,,,
1.0.HCount.1.90usd,Poverty Headcount ($1.90 a day),,LAC Equity Lab,The poverty headcount index measures the propo...,LAC Equity Lab tabulations of SEDLAC (CEDLAS a...,Poverty
1.0.HCount.2.5usd,Poverty Headcount ($2.50 a day),,LAC Equity Lab,The poverty headcount index measures the propo...,LAC Equity Lab tabulations of SEDLAC (CEDLAS a...,Poverty
1.0.HCount.Mid10to50,Middle Class ($10-50 a day) Headcount,,LAC Equity Lab,The poverty headcount index measures the propo...,LAC Equity Lab tabulations of SEDLAC (CEDLAS a...,Poverty
1.0.HCount.Ofcl,Official Moderate Poverty Rate-National,,LAC Equity Lab,The poverty headcount index measures the propo...,LAC Equity Lab tabulations of data from Nation...,Poverty
1.0.HCount.Poor4uds,Poverty Headcount ($4 a day),,LAC Equity Lab,The poverty headcount index measures the propo...,LAC Equity Lab tabulations of SEDLAC (CEDLAS a...,Poverty


receive a list of unique values for topics

In [5]:
indicators['topics'].unique()

array(['Poverty ', '', 'Education ', 'Social Protection & Labor',
       'Economy & Growth', 'Agriculture & Rural Development  ',
       'Agriculture & Rural Development  ,Climate Change',
       'Agriculture & Rural Development  ,Climate Change,Environment ',
       'Climate Change,Urban Development ,Environment ',
       'Climate Change,Environment ',
       'Agriculture & Rural Development  ,Environment ',
       'Urban Development ,Environment ',
       'Economy & Growth,Private Sector,Trade', 'Economy & Growth,Trade',
       'Economy & Growth,Science & Technology ',
       'Economy & Growth,External Debt,Trade',
       'Economy & Growth,Financial Sector ',
       'Economy & Growth,External Debt', 'External Debt',
       'Economy & Growth,External Debt,Aid Effectiveness ',
       'Economy & Growth,Infrastructure ,Trade',
       'Economy & Growth,Financial Sector ,External Debt',
       'Economy & Growth,Financial Sector ,Climate Change',
       'Financial Sector ', 'Aid Effectivene

The topic names above can be used to further narrow your dataset search 

In [6]:
fa = indicators.loc[indicators['topics'] == 'Environment ']
fa.head()

,name,unit,source,sourceNote,sourceOrganization,topics
id,,,,,,
EE.BOD.TOTL.KG,Organic water pollutant (BOD) emissions (kg pe...,,Africa Development Indicators,Emissions of organic water pollutants are meas...,"1998 study by Hemamala Hettige, Muthukumara Ma...",Environment
EG.CFT.ACCS.ZS,Access to clean fuels and technologies for coo...,,World Development Indicators,Access to clean fuels and technologies for coo...,"World Bank, Sustainable Energy for All (SE4ALL...",Environment
EN.ATM.METH.IN.ZS,Energy related methane emissions (% of total),,Africa Development Indicators,Industrial methane emissions are emissions fro...,International Energy Agency (IEA Statistics © ...,Environment
EN.ATM.NOXE.IN.ZS,Nitrous oxide emissions in industrial and ener...,,Africa Development Indicators,Industrial nitrous oxide emissions are emissio...,International Energy Agency (IEA Statistics © ...,Environment
EN.ATM.PM25.MC.T1.ZS,"PM2.5 pollution, population exposed to levels ...",,World Development Indicators,Percent of population exposed to ambient conce...,"Brauer, M. et al. 2017, for the Global Burden ...",Environment


I went through and gathered the Dataset IDs for the datasets needed:

Rainfall: 'AG.LND.PRCP.MM'

AgLand: 'AG.LND.AGRI.K2'

Population: 'SP.POP.TOTL'

ValueAddedAg: 'NV.AGR.TOTL.CD'

In [7]:
#Rainfall
perc = wb.get_series('AG.LND.PRCP.MM', id_or_value='id', simplify_index=True)

In [8]:
#AgLand
land = wb.get_series('AG.LND.AGRI.K2', id_or_value='id', simplify_index=True)

In [9]:
#Population
population = wb.get_series('SP.POP.TOTL', id_or_value='id', simplify_index=True)

In [10]:
#ValueAddedAg
gdp = wb.get_series('NV.AGR.TOTL.CD', id_or_value='id', simplify_index=True)

These are all series, not a dataframe. The next steps takes the rainfall dataset and creates a dataframe

In [11]:
#turn rainfall series into dataframe
df_wb = perc.to_frame()

In [12]:
# change the rainfall column name 
df_wb = df_wb.rename(columns={"AG.LND.PRCP.MM":"rainfall(mm/year)"})

#add the other data in with column names defined 
df_wb['population'] = population 
df_wb['ag_land(sq/km)'] = land
df_wb['ag_gdp'] = gdp

In [13]:
#make index a column and query to only contain countries, not regions
df_wb.reset_index(inplace=True)
#use the key list from above to only show countries, not regions
df_wb = df_wb.loc[df_wb['Country'].isin(key_list)]
df_wb

,Country,Year,rainfall(mm/year),population,ag_land(sq/km),ag_gdp
2989,AFG,1960,NaN,8996967.0,NaN,NaN
2990,AFG,1961,NaN,9169406.0,377000.0,NaN
2991,AFG,1962,327.0,9351442.0,377600.0,NaN
2992,AFG,1963,NaN,9543200.0,378100.0,NaN
2993,AFG,1964,NaN,9744772.0,378730.0,NaN
...,...,...,...,...,...,...
16221,ZWE,2016,NaN,14030338.0,162000.0,1.618000e+09
16222,ZWE,2017,657.0,14236599.0,162000.0,1.466750e+09
16223,ZWE,2018,NaN,14438812.0,162000.0,9.906285e+08
16224,ZWE,2019,NaN,14645473.0,NaN,NaN


In [14]:
df_wb = pd.merge(df_wb, df_key, left_on='Country', right_on='id')

In [41]:
df_wb = df_wb.drop(['Country', 'id'], axis =1)
df_wb.head()

,Year,rainfall(mm/year),population,ag_land(sq/km),ag_gdp,country
0,1960,NaN,8996967.0,NaN,NaN,Afghanistan
1,1961,NaN,9169406.0,377000.0,NaN,Afghanistan
2,1962,327.0,9351442.0,377600.0,NaN,Afghanistan
3,1963,NaN,9543200.0,378100.0,NaN,Afghanistan
4,1964,NaN,9744772.0,378730.0,NaN,Afghanistan


# FAO
Extract the following datasets from manually download: Crop, Yield, Vegetation Index, and Pesticide Use

url where data will be extracted

In [16]:
fao_json = 'http://fenixservices.fao.org/faostat/static/bulkdownloads/datasets_E.json'

send a request to start searching json

In [17]:
response = requests.get(fao_json)
data = response.json()
data

{'Datasets': {'-xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
  'Dataset': [{'DatasetCode': 'AE',
    'DatasetName': 'Discontinued archives and data series: ASTI-Expenditures',
    'Topic': 'All government and nonprofit agencies involved in agricultural research in over 80 low- and middle-income countries. Spending for higher education agencies is estimated in most countries assuming that average spending per researcher at higher education agencies is the same as spending per researcher at government and nonprofit agencies. ASTI is currently exploring ways to more accurately capture agricultural research spending by universities.Private for-profit agencies are not included in ASTI datasets.',
    'DatasetDescription': 'ASTI collects primary time-series data on agricultural research capacity and spending levels through national survey rounds in over 80 low-and middle-income countries. Data collection is carried out by country focal points, who distribute survey forms to all a

Get list of dataset names

In [18]:
for x in range(62):
    print(data['Datasets']['Dataset'][x]['DatasetName'])

Discontinued archives and data series: ASTI-Expenditures
Discontinued archives and data series: ASTI-Researchers
Food Balance: Commodity Balances (non-food)
Investment: Country Investment Statistics Profile
Prices: Consumer Price Indices
Macro-Economic Indicators: Capital Stock
Investment: Development Flows to Agriculture
Land, Inputs and Sustainability: Fertilizers indicators
Climate Change: Emissions intensities
Land, Inputs and Sustainability: Livestock Patterns
Land, Inputs and Sustainability: Land use indicators
Climate Change: Emissions shares
Land, Inputs and Sustainability: Livestock Manure
Land, Inputs and Sustainability: Pesticides indicators
Land, Inputs and Sustainability: Soil nutrient budget
Climate Change: Temperature change
Discontinued archives and data series: Food Aid Shipments (WFP)
Food Balance: Food Balances (2014-)
Food Balance: Food Balances (-2013, old methodology and population)
Investment: Foreign Direct Investment (FDI)
Forestry: Forestry Production and Trad

I went through and gathered the location for the datasets needed:

Yield: [47]

Pesticide Use: [7]

Temperature Change: [15]

In [19]:
#yield file location
yiel = data['Datasets']['Dataset'][47]['FileLocation']
yiel

'http://fenixservices.fao.org/faostat/static/bulkdownloads/Production_Crops_Livestock_E_All_Data_(Normalized).zip'

In [20]:
#request the zip file 
yiel_r = requests.get(yiel, stream=True)

#this is the name of the zip file will be downloaded in local jupyter notebook folder
local_file_yiel = 'yiel_zip.zip'

with open(local_file_yiel, 'wb') as fd:
    for chunk in yiel_r.iter_content(chunk_size=128):
        fd.write(chunk)

In [21]:
#create a zipfile object from created zip above and extract the contents to local directory
with ZipFile(local_file_yiel, 'r') as zipObj:
    zipObj.extractall()

Crop Yield Dataset

In [22]:
df_yiel = pd.read_csv('Production_Crops_Livestock_E_All_Data_(Normalized).csv')
df_yiel.head()

,Area Code,Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag
0,2,Afghanistan,221,"Almonds, with shell",5312,Area harvested,1975,1975,ha,0.0,F
1,2,Afghanistan,221,"Almonds, with shell",5312,Area harvested,1976,1976,ha,5900.0,F
2,2,Afghanistan,221,"Almonds, with shell",5312,Area harvested,1977,1977,ha,6000.0,F
3,2,Afghanistan,221,"Almonds, with shell",5312,Area harvested,1978,1978,ha,6000.0,F
4,2,Afghanistan,221,"Almonds, with shell",5312,Area harvested,1979,1979,ha,6000.0,F


In [23]:
#receieve a list of crops
crops = df_yiel['Item'].unique()
crops

array(['Almonds, with shell', 'Anise, badian, fennel, coriander',
       'Apples', 'Apricots', 'Asses', 'Barley', 'Berries nes',
       'Butter and ghee, sheep milk', 'Butter, cow milk', 'Camels',
       'Cattle', 'Cheese, goat milk', 'Cheese, sheep milk', 'Chickens',
       'Cotton lint', 'Cottonseed', 'Eggs, hen, in shell',
       'Eggs, hen, in shell (number)', 'Fat, camels', 'Fat, cattle',
       'Fat, goats', 'Fat, sheep', 'Figs', 'Fruit, citrus nes',
       'Fruit, fresh nes', 'Fruit, stone nes', 'Goats', 'Grapes',
       'Hides, cattle, fresh', 'Honey, natural', 'Horses', 'Linseed',
       'Maize', 'Meat, camel', 'Meat, cattle', 'Meat, chicken',
       'Meat, game', 'Meat, goat', 'Meat, sheep',
       'Melons, other (inc.cantaloupes)', 'Milk, skimmed cow',
       'Milk, whole fresh camel', 'Milk, whole fresh cow',
       'Milk, whole fresh goat', 'Milk, whole fresh sheep', 'Millet',
       'Molasses', 'Mules', 'Nuts nes', 'Offals, edible, camels',
       'Offals, edible, cattle'

In [24]:
#reduce the dataset to only include the following crops, delete irrelevant columns, and change names 
df_yield = df_yiel.loc[df_yiel['Item'].isin(['Rice, paddy', 'Potatoes', 'Yams', 'Soybeans', 'Wheat', 'Maize', 'Sorghum', 'Cassava'])]
df_yield = df_yield.drop(['Area Code', 'Item Code', 'Element Code', 'Year Code', 'Flag', 'Element'], axis =1)
df_yield.rename(columns = {'Value':'yield(tonnes)'}, inplace = True)
df_yield = df_yield.loc[df_yield['Unit'] == 'tonnes']
df_yield

,Area,Item,Year,Unit,yield(tonnes)
3580,Afghanistan,Maize,1961,tonnes,700000.0
3581,Afghanistan,Maize,1962,tonnes,700000.0
3582,Afghanistan,Maize,1963,tonnes,713000.0
3583,Afghanistan,Maize,1964,tonnes,720000.0
3584,Afghanistan,Maize,1965,tonnes,720000.0
...,...,...,...,...,...
3807803,Net Food Importing Developing Countries,Yams,2015,tonnes,12709678.0
3807804,Net Food Importing Developing Countries,Yams,2016,tonnes,13493232.0
3807805,Net Food Importing Developing Countries,Yams,2017,tonnes,13905513.0
3807806,Net Food Importing Developing Countries,Yams,2018,tonnes,13626072.0


Pesticide/Fertilizer Dataset

In [25]:
#pesticide file location
pest = data['Datasets']['Dataset'][7]['FileLocation']
pest

'http://fenixservices.fao.org/faostat/static/bulkdownloads/Environment_Fertilizers_E_All_Data_(Normalized).zip'

In [26]:
#request the zip file 
pest_r = requests.get(pest, stream=True)

#this is the name of the zip file will be downloaded in local jupyter notebook folder
local_file_pest = 'pest_zip.zip'

with open(local_file_pest, 'wb') as fd:
    for chunk in pest_r.iter_content(chunk_size=128):
        fd.write(chunk)

In [27]:
#create a zipfile object from created zip above and extract the contents to local directory
with ZipFile(local_file_pest, 'r') as zipObj:
    zipObj.extractall()

In [28]:
df_pest = pd.read_csv('Environment_Fertilizers_E_All_Data_(Normalized).csv')
df_pest = df_pest.drop(['Area Code', 'Item Code', 'Element Code', 'Year Code', 'Flag', 'Element', 'Item'], axis =1)
df_pest.head()

,Area,Year,Unit,Value
0,Afghanistan,1961,kg/ha,0.13
1,Afghanistan,1962,kg/ha,0.13
2,Afghanistan,1963,kg/ha,0.13
3,Afghanistan,1964,kg/ha,0.13
4,Afghanistan,1965,kg/ha,0.13


In [29]:
#Total amount of pesticides used each year 
df_pest = df_pest.groupby(['Area', 'Year']).agg({'Value': 'sum'})
df_pest.reset_index(inplace=True)
df_pest.rename(columns = {'Value':'pestUse(kg/ha)'}, inplace = True)
df_pest

,Area,Year,pestUse(kg/ha)
0,Afghanistan,1961,0.14
1,Afghanistan,1962,0.14
2,Afghanistan,1963,0.14
3,Afghanistan,1964,0.14
4,Afghanistan,1965,0.14
...,...,...,...
10489,Zimbabwe,2015,19.17
10490,Zimbabwe,2016,31.78
10491,Zimbabwe,2017,35.71
10492,Zimbabwe,2018,32.39


Temperature Change Dataset

In [30]:
#temp change file location
temp = data['Datasets']['Dataset'][15]['FileLocation']
temp

'http://fenixservices.fao.org/faostat/static/bulkdownloads/Environment_Temperature_change_E_All_Data_(Normalized).zip'

In [31]:
#request the zip file 
temp_r = requests.get(temp, stream=True)

#this is the name of the zip file will be downloaded in local jupyter notebook folder
local_file_temp = 'temp_zip.zip'

with open(local_file_temp, 'wb') as fd:
    for chunk in temp_r.iter_content(chunk_size=128):
        fd.write(chunk)

In [32]:
#create a zipfile object from created zip above and extract the contents to local directory
with ZipFile(local_file_temp, 'r') as zipObj:
    zipObj.extractall()

In [33]:
df_temp = pd.read_csv('Environment_Temperature_change_E_All_Data_(Normalized).csv')
df_temp.head()

,Area Code,Area,Months Code,Months,Element Code,Element,Year Code,Year,Unit,Value,Flag
0,2,Afghanistan,7001,January,7271,Temperature change,1961,1961,�C,0.746,Fc
1,2,Afghanistan,7001,January,7271,Temperature change,1962,1962,�C,0.009,Fc
2,2,Afghanistan,7001,January,7271,Temperature change,1963,1963,�C,2.695,Fc
3,2,Afghanistan,7001,January,7271,Temperature change,1964,1964,�C,-5.277,Fc
4,2,Afghanistan,7001,January,7271,Temperature change,1965,1965,�C,1.827,Fc


In [34]:
#sum the temp change for each year then average that number 
df_temp = df_temp.groupby(['Area', 'Year']).agg({'Value': 'sum'})
df_temp.reset_index(inplace=True)
df_temp['Value'] = df_temp['Value']/12
df_temp.rename(columns = {'Value':'tempChange(C)'}, inplace = True)
df_temp

,Area,Year,tempChange(C)
0,Afghanistan,1961,1.659750
1,Afghanistan,1962,1.332000
2,Afghanistan,1963,2.886083
3,Afghanistan,1964,0.326083
4,Afghanistan,1965,1.539583
...,...,...,...
15800,Zimbabwe,2016,2.904667
15801,Zimbabwe,2017,1.539667
15802,Zimbabwe,2018,2.082167
15803,Zimbabwe,2019,2.956333


In [35]:
df_temp.head()

,Area,Year,tempChange(C)
0,Afghanistan,1961,1.659750
1,Afghanistan,1962,1.332000
2,Afghanistan,1963,2.886083
3,Afghanistan,1964,0.326083
4,Afghanistan,1965,1.539583


Merge the temperture change and pesticide/fertilizer dataset

In [36]:
df_fao = pd.merge(df_temp, df_pest, left_on=['Area', 'Year'], right_on=['Area', 'Year'], how='outer')
df_fao

,Area,Year,tempChange(C),pestUse(kg/ha)
0,Afghanistan,1961,1.659750,0.14
1,Afghanistan,1962,1.332000,0.14
2,Afghanistan,1963,2.886083,0.14
3,Afghanistan,1964,0.326083,0.14
4,Afghanistan,1965,1.539583,0.14
...,...,...,...,...
15822,Bermuda,2015,NaN,285.50
15823,Bermuda,2016,NaN,177.21
15824,Bermuda,2017,NaN,181.47
15825,Bermuda,2018,NaN,214.93


Combine the yield dataset with the previously created dataset

In [37]:
df_fao = pd.merge(df_yield, df_fao, left_on=['Area', 'Year'], right_on=['Area', 'Year'], how='outer')
df_fao

,Area,Item,Year,Unit,yield(tonnes),tempChange(C),pestUse(kg/ha)
0,Afghanistan,Maize,1961,tonnes,700000.0,1.65975,0.14
1,Afghanistan,Potatoes,1961,tonnes,130000.0,1.65975,0.14
2,Afghanistan,"Rice, paddy",1961,tonnes,319000.0,1.65975,0.14
3,Afghanistan,Wheat,1961,tonnes,2279000.0,1.65975,0.14
4,Afghanistan,Maize,1962,tonnes,700000.0,1.33200,0.14
...,...,...,...,...,...,...,...
69139,Bermuda,NaN,2015,NaN,NaN,NaN,285.50
69140,Bermuda,NaN,2016,NaN,NaN,NaN,177.21
69141,Bermuda,NaN,2017,NaN,NaN,NaN,181.47
69142,Bermuda,NaN,2018,NaN,NaN,NaN,214.93


# Combine FAO and World Bank data

In [40]:
df_master = pd.merge(df_wb, df_fao, left_on=['country', 'Year'], right_on=['Area', 'Year'], how='outer')

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat